In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[11],9
commitnumber,1013f91,a06b618
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[5]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312        1 -6.702704e-04                    1 -0.000032
0.000750        2 -6.699875e-04                    2 -0.000034
0.001052        3 -6.689541e-04                    3 -0.000029
0.001476        4 -6.676074e-04                    4 -0.000027
0.002070        5 -6.660580e-04                    5 -0.000025
0.002904        6 -6.644343e-04                    6 -0.000023
0.004074        7 -6.628377e-04                    7 -0.000022
0.005714        8 -6.610948e-04                    8 -0.000020
0.008015        9 -6.595067e-04                    9 -0.000018
0.011243       10 -6.579689e-04                   10 -0.000015
0.015771       11 -6.570496e-04                   11 -0.000013
0.022122       12 -6.568870e-04                   12 -0.000011
0.031031       13 -6.578162e-04                   13 -0.000008
0.043528       14 -6.601551e-04                   14 -0.000006
0.061057       15 -6.548656e-04                   15  0.000002
0.085645       16 -6.109366e-04                   16  0.000032
0.120136       17 -4.915039e-04                   17  0.000097
0.168516       18 -2.811832e-04                   18  0.000203
0.236378       19  5.629998e-05                   19  0.000367
0.331549       20  5.728185e-04                   20  0.000618
0.465100       21  1.339775e-03                   21  0.000996
0.652400       22  2.453328e-03                   22  0.001552
0.915100       23  3.635652e-03                   23  0.002164
1.283650       24  3.613053e-03                   24  0.002226
1.800600       25  2.495162e-03                   25  0.001735
2.525700       26  1.495763e-03                   26  0.001237
3.542800       27  8.300265e-04                   27  0.000854
4.969550       28  4.328873e-04                   28  0.000577
6.970850       29  2.221521e-04                   29  0.000387
9.778100       30  1.151002e-04                   30  0.000255
13.715850      31  5.448747e-05                   31  0.000158
19.239350      32  1.799927e-05                   32  0.000087
26.987250      33  3.654192e-06                   33  0.000041
37.855300      34 -4.652112e-07                   34  0.000012
53.100050      35 -9.582664e-06                   35 -0.000013
73.887500      36 -2.361337e-05                   36 -0.000036
97.662500      37 -3.341440e-05                   37 -0.000050
121.437500     38 -3.944783e-05                   38 -0.000059
145.212500     39 -4.915230e-05                   39 -0.000068
168.987500     40 -5.901181e-05                   40 -0.000076
192.762500     41 -7.455693e-05                   41 -0.000103
216.537500     42 -1.064638e-04                   42 -0.000189
240.312500     43 -1.220898e-04                   43 -0.000304
264.087500     44 -1.569970e-04                   44 -0.000505
287.862500     45 -2.566387e-04                   45 -0.000674
311.637500     46 -3.230123e-04                   46 -0.000667
335.412500     47 -2.668091e-04                   47 -0.000622
359.187500     48 -8.240388e-05                   48 -0.000654
382.962500     49  2.318334e-04                   49 -0.000724
406.737500     50  6.805777e-04                   50 -0.000713
430.512500     51  1.253684e-03                   51 -0.000461
454.287500     52  1.925891e-03                   52  0.000135
478.062500     53  2.726550e-03                   53  0.001221
501.837500     54  3.676744e-03                   54  0.002630
525.612500     55  4.786407e-03                   55  0.004371
549.387500     56  6.080320e-03                   56  0.006360
573.162500     57  7.611592e-03                   57  0.008529
596.937500     58  9.679464e-03                   58  0.011129
620.712500     59  1.181076e-02                   59  0.013479
644.487500     60  1.393084e-02                   60  0.015292
668.262500     61  1.607643e-02              

# Fluxes by Layer

CRD                            CLIRAD                \
                       flug          fldg     fnetg      flug          fldg   
pressure    level                                                             
0.000000    1     -5.098632  0.000000e+00 -5.098632 -4.873496  7.005192e-09   
0.000624    2     -5.098632  5.618312e-09 -5.098632 -4.873496  8.675398e-09   
0.000876    3     -5.098632  7.921962e-09 -5.098632 -4.873496  9.489031e-09   
0.001229    4     -5.098632  1.127727e-08 -5.098632 -4.873496  1.073326e-08   
0.001723    5     -5.098632  1.617629e-08 -5.098632 -4.873496  1.247930e-08   
0.002417    6     -5.098632  2.332701e-08 -5.098632 -4.873496  1.489706e-08   
0.003391    7     -5.098632  3.376440e-08 -5.098632 -4.873496  1.833635e-08   
0.004757    8     -5.098632  4.898635e-08 -5.098632 -4.873496  2.321516e-08   
0.006672    9     -5.098632  7.117703e-08 -5.098632 -4.873496  3.025313e-08   
0.009359    10    -5.098632  1.035146e-07 -5.098632 -4.873496  4.037260e-08   
0.013128    11    -5.098633  1.505885e-07 -5.098633 -4.873496  5.507810e-08   
0.018415    12    -5.098633  2.190148e-07 -5.098633 -4.873496  7.643487e-08   
0.025830    13    -5.098634  3.183752e-07 -5.098634 -4.873496  1.074760e-07   
0.036232    14    -5.098635  4.623886e-07 -5.098634 -4.873496  1.527429e-07   
0.050823    15    -5.098636  6.712033e-07 -5.098636 -4.873496  2.188709e-07   
0.071291    16    -5.098638  9.989392e-07 -5.098637 -4.873496  3.250115e-07   
0.100000    17    -5.098641  1.626744e-06 -5.098639 -4.873497  5.351468e-07   
0.140271    18    -5.098645  2.978889e-06 -5.098642 -4.873497  1.001658e-06   
0.196760    19    -5.098649  5.859960e-06 -5.098643 -4.873496  2.032578e-06   
0.275997    20    -5.098655  1.188057e-05 -5.098643 -4.873495  4.283229e-06   
0.387100    21    -5.098660  2.421211e-05 -5.098635 -4.873492  9.139343e-06   
0.543100    22    -5.098660  4.898738e-05 -5.098611 -4.873484  1.952371e-05   
0.761700    23    -5.098645  9.748603e-05 -5.098547 -4.873465  4.137090e-05   
1.068500    24    -5.098598  1.826684e-04 -5.098415 -4.873429  8.319525e-05   
1.498800    25    -5.098521  2.904318e-04 -5.098231 -4.873375  1.432763e-04   
2.102400    26    -5.098439  3.864776e-04 -5.098052 -4.873317  2.095636e-04   
2.949000    27    -5.098360  4.578924e-04 -5.097902 -4.873261  2.770199e-04   
4.136600    28    -5.098289  5.030207e-04 -5.097786 -4.873208  3.438985e-04   
5.802500    29    -5.098225  5.250968e-04 -5.097700 -4.873159  4.094811e-04   
8.139200    30    -5.098170  5.308380e-04 -5.097639 -4.873117  4.740284e-04   
11.417000   31    -5.098121  5.267877e-04 -5.097594 -4.873081  5.376276e-04   
16.014700   32    -5.098082  5.172109e-04 -5.097564 -4.873056  5.988212e-04   
22.464000   33    -5.098057  5.064647e-04 -5.097551 -4.873047  6.559579e-04   
31.510500   34    -5.098050  5.033719e-04 -5.097547 -4.873058  7.105795e-04   
44.200100   35    -5.098064  5.163816e-04 -5.097547 -4.873095  7.654849e-04   
62.000000   36    -5.098114  5.468004e-04 -5.097567 -4.873177  8.193496e-04   
85.775000   37    -5.098229  5.945743e-04 -5.097634 -4.873325  8.676236e-04   
109.550000  38    -5.098382  6.533053e-04 -5.097728 -4.873506  9.067538e-04   
133.325000  39    -5.098564  7.251528e-04 -5.097839 -4.873713  9.464858e-04   
157.100000  40    -5.098782  8.039059e-04 -5.097978 -4.873946  9.873719e-04   
180.875000  41    -5.099044  8.999134e-04 -5.098144 -4.874211  1.037805e-03   
204.650000  42    -5.099465  1.111467e-03 -5.098354 -4.874629  1.163736e-03   
228.425000  43    -5.100509  1.854841e-03 -5.098654 -4.875692  1.693112e-03   
252.200000  44    -5.102678  3.680897e-03 -5.098998 -4.877957  3.101160e-03   
275.975000  45    -5.106697  7.256927e-03 -5.099440 -4.882222  5.943542e-03   
299.750000  46    -5.113538  1.337575e-02 -5.100163 -4.889552  1.137430e-02   
323.525000  47    -5.122907  2.183442e-02 -5.101072 -4.899650  1.959397e-02   
347.300000  48    -5.134752  3.292813e-02 -5.101824 -4.912484  3.067520e-0

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')